In [ ]:
import pickle

import numpy as np

import matplotlib.pyplot as plt
from cycler import cycler
from coffea import hist, util

from utils.plotting import plotWithRatio

In [ ]:
outputMC = util.load('ttgammaMC.coffea')
outputData = util.load('ttgammaData.coffea')

In [ ]:
outputMC

In [ ]:
grouping= {'$t\overline{t}+\gamma$': ['TTGamma_Dilepton','TTGamma_SingleLept','TTGamma_Hadronic'],
        '$t\overline{t}$'  : ['TTbarPowheg_Dilepton', 'TTbarPowheg_Semilept', 'TTbarPowheg_Hadronic'],
        'Single top':['ST_s_channel', 'ST_tW_channel', 'ST_tbarW_channel', 'ST_tbar_channel', 'ST_t_channel'],
        'W+jets':['W1jets', 'W2jets', 'W3jets', 'W4jets'],
        'Z+jets'  : ['DYjetsM10to50', 'DYjetsM50'],
        'W+$\gamma$' : ['WGamma_01J_5f'],
        'Z+$\gamma$' : ['ZGamma_01J_5f_lowMass'],
        'TTV'    : ['TTWtoLNu','TTWtoQQ','TTZtoLL'],
        'QCD' : [ 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_Pt20to30_Ele', 'QCD_Pt30to50_Ele', 'QCD_Pt50to80_Ele', 'QCD_Pt80to120_Ele', 'QCD_Pt120to170_Ele', 'QCD_Pt170to300_Ele', 'QCD_Pt300toInf_Ele', 'QCD_Pt20to30_Mu', 'QCD_Pt30to50_Mu', 'QCD_Pt50to80_Mu', 'QCD_Pt80to120_Mu', 'QCD_Pt120to170_Mu', 'QCD_Pt170to300_Mu', 'QCD_Pt300to470_Mu', 'QCD_Pt470to600_Mu', 'QCD_Pt600to800_Mu', 'QCD_Pt800to1000_Mu', 'QCD_Pt1000toInf_Mu']
  }

In [ ]:
h = outputMC['photon_chIso'].sum("dataset").sum("lepFlavor")#.rebin('chIso',hist.Bin("chIso",'ChIso',np.arange(0,20.001,.2)))
hData = outputData['photon_chIso'].sum("dataset").sum("category").sum("lepFlavor")#.rebin('chIso',hist.Bin("chIso",'ChIso',np.arange(0,20.001,.2)))

plotWithRatio(h, hData, overlay='category', invertStack=True, xRange=[-.1,20], yRange=[1e-5,None], logY=True)
plotWithRatio(h, None, overlay='category', stacked=False, density=True, xRange=[-.1,20], yRange=[1e-5,None], logY=True,leg=None)



In [ ]:

h = outputMC['photon_chIso'].sum("category").sum("lepFlavor")#.rebin('chIso',hist.Bin("chIso",'ChIso',np.arange(0,20.001,.2)))
hData = outputData['photon_chIso'].sum("category").sum('dataset').sum("lepFlavor")#.rebin('chIso',hist.Bin("chIso",'ChIso',np.arange(0,20.001,.2)))

plotWithRatio(h.group('dataset',hist.Cat(r'dataset',r'Samples'),grouping), hData, overlay='dataset', invertStack=True, xRange=[-.1,5], yRange=[1e-5,None], logY=True)



In [ ]:
h = outputMC['M3'].sum("lepFlavor").rebin('M3',hist.Bin("M3",r"$M_3$ [GeV]",np.arange(0,500.001,10)))
hData = outputData['M3'].sum("lepFlavor").sum('category').sum('dataset').rebin('M3',hist.Bin("M3",r"$M_3$ [GeV]",np.arange(0,500.001,10)))

plotWithRatio(h.sum("category").group('dataset',hist.Cat(r'dataset',r'Samples'),grouping), hData, overlay='dataset', invertStack=True)
plotWithRatio(h.sum("dataset"), hData, overlay='category', invertStack=True)



In [ ]:

h = outputMC['photon_pt'].group('dataset',hist.Cat(r'dataset',r'Samples'),grouping).sum('category').sum("lepFlavor")
h = h.rebin('pt',hist.Bin('pt','$p_{T}$',np.arange(0,200.1,5)))

hData = outputData['photon_pt'].sum('dataset').sum('category').sum("lepFlavor")
hData = hData.rebin('pt',hist.Bin('pt','$p_{T}$',np.arange(0,200.1,5)))

plotWithRatio(h, hData, overlay='dataset', invertStack=True)


In [ ]:
h = outputMC['M3Presel'].sum("lepFlavor").rebin('M3',hist.Bin("M3",r"$M_3$ [GeV]",np.arange(0,500.001,10)))
hData = outputData['M3Presel'].sum("lepFlavor").sum('dataset').rebin('M3',hist.Bin("M3",r"$M_3$ [GeV]",np.arange(0,500.001,10)))

plotWithRatio(h.group('dataset',hist.Cat(r'dataset',r'Samples'),grouping), hData, overlay='dataset', invertStack=True)



In [ ]:

h = outputMC['photon_lepton_mass_3j0t'].group('dataset',hist.Cat(r'dataset',r'Samples'),grouping)
h = h.rebin('mass',hist.Bin('mass',r"$m_{\ell\gamma}$ [GeV]",np.arange(0,250.1,5)))

hData = outputData['photon_lepton_mass_3j0t'].sum('dataset').sum('category')
hData = hData.rebin('mass',hist.Bin('mass',r"$m_{\ell\gamma}$ [GeV]",np.arange(0,250.1,5)))

plotWithRatio(h.integrate('lepFlavor',slice(-1.5,0)).sum('category'), hData.integrate('lepFlavor',slice(-1.5,0)), overlay='dataset', invertStack=True)
plotWithRatio(h.integrate('lepFlavor',slice(-1.5,0)).sum('dataset'), hData.integrate('lepFlavor',slice(-1.5,0)), overlay='category', invertStack=True)

plotWithRatio(h.integrate('lepFlavor',slice(0,1.5)).sum('category'), hData.integrate('lepFlavor',slice(0,1.5)), overlay='dataset', invertStack=True)
plotWithRatio(h.integrate('lepFlavor',slice(0,1.5)).sum('dataset'), hData.integrate('lepFlavor',slice(0,1.5)), overlay='category', invertStack=True)
